In [1]:
import numpy as np
import pandas as pd
from flask import Flask, redirect, url_for, request
from keras.models import model_from_json
import json

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from subprocess import check_output
diff=pd.read_csv('diffsydiw.csv')
sym=pd.read_csv('sym_t.csv')
dia=pd.read_csv('dia_t.csv')
print(diff.head())

   syd    did  wei
0  1.0  163.0  2.0
1  1.0  164.0  2.0
2  1.0  165.0  1.0
3  1.0  187.0  2.0
4  1.0  306.0  2.0


In [3]:
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm


from scipy.sparse import coo_matrix, csr_matrix
def read_data(filename):
    data = pd.read_csv(filename,usecols=[0,1,2],names=['user', 'song','plays'],skiprows=1)
    data=data.dropna(axis=0, how='any')  #drop nan
    data['plays']=data['plays']+1
    print(data.head())
    # map each song and user to a unique numeric value
    data['user'] = data['user'].astype("category")
    data['song'] = data['song'].astype("category")

    # create a sparse matrix of all the users/plays
    plays = coo_matrix((data['plays'].astype(float),
                       (data['song'].cat.codes.copy(),
                        data['user'].cat.codes.copy())))
    
    return data, plays,data.groupby(['song']).plays.sum(),data['user'].cat.codes.copy()

In [4]:
data,matrix,songsd,user=read_data('diffsydiw.csv')
data.head(10)

   user   song  plays
0   1.0  163.0    3.0
1   1.0  164.0    3.0
2   1.0  165.0    2.0
3   1.0  187.0    3.0
4   1.0  306.0    3.0


,user,song,plays
0,1.0,163.0,3.0
1,1.0,164.0,3.0
2,1.0,165.0,2.0
3,1.0,187.0,3.0
4,1.0,306.0,3.0
5,1.0,307.0,2.0
6,1.0,308.0,3.0
7,1.0,309.0,3.0
8,1.0,354.0,2.0
9,1.0,401.0,2.0


In [5]:
from sklearn.preprocessing import normalize


def cosine(plays):
    normalized = normalize(plays)
    return normalized.dot(normalized.T)


def bhattacharya(plays):
    plays.data = np.sqrt(plays.data)
    return cosine(plays)


def ochiai(plays):
    plays = csr_matrix(plays)
    plays.data = np.ones(len(plays.data))
    return cosine(plays)
def bm25_weight(data, K1=1.2, B=0.8):
    """ Weighs each row of the matrix data by BM25 weighting """
    # calculate idf per term (user)
    N = float(data.shape[0])
    idf = np.log(N / (1 + np.bincount(data.col)))

    # calculate length_norm per document (artist)
    row_sums = np.squeeze(np.asarray(data.sum(1)))
    average_length = row_sums.sum() / N
    length_norm = (1.0 - B) + B * row_sums / average_length

    # weight matrix rows by bm25
    ret = coo_matrix(data)
    ret.data = ret.data * (K1 + 1.0) / (K1 * length_norm[ret.row] + ret.data) * idf[ret.col]
    return ret

def get_largest(row, N=10):
    if N >= row.nnz:
        best = zip(row.data, row.indices)
    else:
        ind = np.argpartition(row.data, -N)[-N:]
        best = zip(row.data[ind], row.indices[ind])
    return sorted(best, reverse=True)
def bm25(plays):
    plays = bm25_weight(plays)
    return plays.dot(plays.T)

def calculate_similar_artists(similarity, artists, artistid):
    neighbours = similarity[artistid]
    top = get_largest(neighbours)
    return [(artists[other], score, i) for i, (score, other) in enumerate(top)]


#songsd = dict(enumerate(data['song'].cat.categories))
user_count = data.groupby('user').size()
#to_generate = sorted(list(songsd), key=lambda x: -user_count[x])

similarity = bm25_weight(matrix)

In [6]:
#print(sym)
sym[sym['syd'].isin(list(songsd.index))]

,syd,symptom
0,1,Upper abdominal pain
1,2,Lower abdominal pain
2,3,Abscess (Collection of pus)
3,4,Alcohol abuse
4,5,Anxiety (Nervousness)
5,6,Arm ache or pain
6,7,Back ache or pain
7,8,Bleeding tendency
8,9,Blood in vomit
10,11,Pain or soreness of breast


In [7]:
from scipy.sparse.linalg import svds

Ur, Si, VTr = svds(bm25_weight(coo_matrix(matrix)), k=100)
print(Ur.shape, Si.shape, VTr.shape,user.shape,matrix.shape,data.shape,songsd.shape,user_count.shape)
VTr=pd.DataFrame(VTr)

(1145, 100) (100,) (100, 272) (5371,) (1145, 272) (5371, 3) (1145,) (272,)


In [8]:
from sklearn.metrics.pairwise import cosine_similarity
Sddf=pd.DataFrame(cosine_similarity(Ur,VTr.T),columns=user_count.index,index=list(songsd.index)) 
Sddf.to_csv('Sddf.csv')

Sydi=pd.DataFrame(cosine_similarity(Ur,VTr.T))

In [9]:
booknr=29 #symptoom4
print('Symptom',sym[sym['syd']==booknr])
print('top 7 related disease probability') #,Sddf[booknr].sort_values(ascending=False))
print()
lijst= Sddf[booknr].sort_values(ascending=False).index 
for xi in lijst[:7]:
    print(xi,dia[dia['did']==xi].diagnose.values)

Symptom     syd symptom
28   29   Fever
top 7 related disease probability

800.0 ['Vaccination']
904.0 ['Rocky mountain spotted fever\x0bbacterial infection spread by ticks']
567.0 ['Peritonitis\x0babdominal cavity inflammation']
1495.0 ['Anal fistula']
1441.0 ['Lymphadenitis']
1455.0 ['Neutropenia']
1329.0 ['Neuroblastoma']


In [ ]:
app=Flask(__name__)
@app.route('/analyzer/')
def analyzer():
    a=int(request.args.get('ids'))
    lijst= Sddf[a].sort_values(ascending=False).index 
    x1=[]
    for xi in lijst[:7]:
        x1.append(dia[dia['did']==xi].diagnose.values)
    x1=pd.Series(x1).to_json(orient='values')
    return json.dumps(x1)
#jsonify(range(x))
@app.route('/symptom',methods=['POST','GET']) 
def symptom():
    if request.method=='POST':
        i=int(request.form['id'])
        return redirect(url_for('analyzer',ids=i))
    else:
        i=int(request.args.get('id'))
        return redirect(url_for('analyzer',ids=i))
if __name__=='__main__':
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Nov/2018 00:58:04] "POST /symptom HTTP/1.1" 302 -
127.0.0.1 - - [21/Nov/2018 00:58:04] "GET /analyzer/?ids=4 HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2018 00:58:25] "POST /symptom HTTP/1.1" 302 -
127.0.0.1 - - [21/Nov/2018 00:58:25] "GET /analyzer/?ids=1 HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2018 00:59:17] "POST /symptom HTTP/1.1" 302 -
[2018-11-21 00:59:17,108] ERROR in app: Exception on /analyzer/ [GET]
Traceback (most recent call last):
  File "D:\Anaconda\lib\site-packages\pandas\core\indexes\base.py", line 3063, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 140, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 162, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 379, in pandas._libs.hashtable.Float64HashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 385, in pandas._l

In [ ]:
lijst=list(lijst[:3])
vectori=[0 for x in range(0,len(user_count))]
for xi in lijst:
    #ii=songsd.index(xi)
    vectori+=Sddf.loc[xi]
vect=pd.DataFrame(vectori)
vect.columns=['para']
vect=vect[vect>1].dropna(axis=0)
print('other symptoms',vect/3)
print(sym[sym.syd.isin(list(vect.index))])